In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install janome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os
import gc
import random
from collections import defaultdict, Counter
from typing import List, Dict
import joblib
import pickle

from tqdm import tqdm
import numpy as np
import polars as pl
import pandas as pd
from janome.tokenizer import Tokenizer

In [5]:
product = pl.read_csv("/gdrive/MyDrive/amazon_kdd_2023/data/raw/products_train.csv")

# Priceの異常値削除

In [6]:
product = product.with_columns(
    pl.when(product['price'] == 40000000.07).then(None).otherwise(product['price']).alias("price")
)

# 文章の正規化(JP)

In [7]:
product_jp = product.filter(pl.col("locale") == "JP")

## 正規化処理

In [8]:
COLS = [
    "title",
    "brand",
    "desc",
]

In [9]:
from __future__ import unicode_literals
import re
import unicodedata

# based on https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja
def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

# def remove_extra_spaces(s):
#     s = re.sub('[ 　]+', ' ', s)
#     blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
#                       '\u3040-\u309F',  # HIRAGANA
#                       '\u30A0-\u30FF',  # KATAKANA
#                       '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
#                       '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
#                       ))
#     basic_latin = '\u0000-\u007F'

#     def remove_space_between(cls1, cls2, s):
#         p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
#         while p.search(s):
#             s = p.sub(r'\1\2', s)
#         return s

#     s = remove_space_between(blocks, blocks, s)
#     s = remove_space_between(blocks, basic_latin, s)
#     s = remove_space_between(basic_latin, blocks, s)
#     return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]', '〜', s)  # normalize tildes
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    # s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

In [10]:
for col in tqdm(COLS):
    texts = product_jp[col].to_list()
    texts_normalized = []
    for text in texts:
        if text:
            texts_normalized.append(normalize_neologd(text))
        else:
            texts_normalized.append(text)
    product_jp = product_jp.with_columns(
        pl.Series(name=col, values=texts_normalized)
    )

100%|██████████| 3/3 [01:02<00:00, 20.70s/it]


## 形態素解析

In [11]:
COLS = [
    "desc",
]

In [13]:
t = Tokenizer()

for col in COLS:
    texts = product_jp[col].to_list()
    texts_wakati = []
    for text in tqdm(texts):
        if text:
            texts_wakati.append(" ".join(str(word) for word in t.tokenize(text, wakati=True)))
        else:
            texts_wakati.append(text)
    product_jp = product_jp.with_columns(
        pl.Series(name=col, values=texts_wakati)
    )

100%|██████████| 395009/395009 [30:53<00:00, 213.10it/s]


In [14]:
product = pl.concat([
    product.filter(pl.col("locale") != "JP"),
    product_jp,
])

In [15]:
product.write_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/preprocessed/common/product_02.parquet")

In [16]:
product_jp.to_pandas()

,id,locale,title,price,brand,color,size,model,material,author,desc
0,B0BCFTG541,JP,ワタシガモテナイノハドウカンガエテモオマエラガワルイ022 (デジタルバンガンガンコミックス...,600.0,スクウェア・エニックス,None,None,None,None,タニガワニコ,None
1,B096F11HV9,JP,"Apple MacBook Air M1 2020(13インチAir,8GB RAM,256...",152000.0,Apple Computer,スペースグレイ,"RAM:8GB/ SSD:256GB/ 8コアCPU,7コアGPU",None,None,None,Touch ID : Touch ID センサー
2,B0178VYR6Q,JP,キユーピー やさしい献立 なめらかごはん 150g×6個【区分4:かまなくてよい】,1111.0,キユーピー,None,150グラム (x 6),None,None,None,カロリー : 79 kcal
3,B09VWM3SCD,JP,Coticam outdoor ランタンシェード LED ライト ランタン キャンプ アウト...,999.0,Coticam outdoor,ブラック,None,None,None,None,※ 微細 な シワ 、 スジ 、 色 ムラ が ある 場合 が あり ます 。 ご 了承 願...
4,B07KFZPKLV,JP,Colorsroom こたつ布団 正方形 コタツ布団 185×185cm コタツふとん 掛け...,4970.0,布団と寝具専門店カラーズ,ボタニカル柄・ベージュ,こたつ布団185×185cm,None,"フランネル,コットン",None,【 ご 家庭 で 丸洗い できる 、 いつも 清潔 】 ウォッシャブル 仕様 だ から 、 ...
...,...,...,...,...,...,...,...,...,...,...,...
395004,B08DNQ687B,JP,【クール・常温選択】にんじん5kg 農薬不使用・化学肥料不使用 わけあり 規格外品 人参...,2080.0,おやさい村,None,5キログラム (x 1),None,None,None,クール 便 で 発送 月曜 から 土曜 まで 14 時 まで の ご 注文 で 当日 発...
395005,B0932DFYZP,JP,Anjuer ウォールハンガー 壁取り付け式コートラック 5連フック 壁掛けフック 収納家具...,1899.0,Anjuer,木色+黒,5連フック,None,竹,None,● 取り付け 簡単 : 簡単 に 組み立て られ 、 すぐ 使用 でき ます 。 おしゃれ ...
395006,B095HJ7NM4,JP,静電気防止 ブレスレット 男女兼用 静電気除去ブレスレット [パチッとくる 静電気 をしっか...,1280.0,PHATHiNG,ブルー,None,None,シリコーン,None,【 スタイリッシュ に 静電気 防止 】 本 製品 の 静電気 防止 ブレス...
395007,B07XJ2TFTX,JP,switch liteカバー スイッチライト ケース TPU素材 一体式 全面保護 耐衝撃 ...,1699.0,CHINFAI,ブラック,None,None,None,None,🎮 【 スタンド 機能 付き 】 スタンド 機能 あり 特殊 ハンドル 設計 は オリジナル...
